In [1]:
import numpy as np
import pickle
import cv2
import os
from os import listdir
from sklearn.preprocessing import LabelBinarizer
from keras.models import Sequential
from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers.core import Activation, Flatten, Dropout, Dense
from keras import backend as K
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam
from keras.preprocessing import image
from keras.preprocessing.image import img_to_array
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

In [2]:
EPOCHS = 25
BS = 32
default_image_size = tuple((256, 256))
image_size = 0
directory_root = '/Users/samya/OneDrive/Desktop/CropDetection'
width=256
height=256
depth=3

In [3]:
def convert_image_to_array(image_dir):
    try:
        image = cv2.imread(image_dir)
        if image is not None :
            image = cv2.resize(image, default_image_size)   
            return img_to_array(image)
        else :
            return np.array([])
    except Exception as e:
        print(f"Error : {e}")
        return None

In [4]:
image_list, label_list = [], []
try:
  print("[INFO] Loading images....")
  root_dir = listdir(directory_root)
      
  for plant_folder in root_dir:
    print(f"[INFO] Processing {plant_folder} ...")
    plant_disease_image_list = listdir(f"{directory_root}/{plant_folder}")

    for image in plant_disease_image_list[:200]:
      image_directory = f"{directory_root}/{plant_folder}/{image}"
      if image_directory.endswith(".jpg") == True or image_directory.endswith(".JPG") == True:
        image_list.append(convert_image_to_array(image_directory))
        label_list.append(plant_folder)
  print("[INFO] Image loading completed")
except Exception as e:
  print(f"Error : {e}")


[INFO] Loading images....
[INFO] Processing Peach___Bacterial_spot ...
[INFO] Processing Peach___healthy-20210217T071520Z-001 ...
[INFO] Processing Pepper__bell___Bacterial_spot ...
[INFO] Processing Pepper__bell___healthy ...
[INFO] Processing Potato___Early_blight ...
[INFO] Processing Potato___healthy ...
[INFO] Processing Potato___Late_blight ...
[INFO] Processing Tomato_Bacterial_spot ...
[INFO] Processing Tomato_Early_blight ...
[INFO] Processing Tomato_healthy ...
[INFO] Processing Tomato_Late_blight ...
[INFO] Processing Tomato_Leaf_Mold ...
[INFO] Processing Tomato_Septoria_leaf_spot ...
[INFO] Processing Tomato_Spider_mites_Two_spotted_spider_mite ...
[INFO] Processing Tomato__Target_Spot ...
[INFO] Processing Tomato__Tomato_mosaic_virus ...
[INFO] Processing Tomato__Tomato_YellowLeaf__Curl_Virus ...
[INFO] Image loading completed


In [5]:
image_size = len(image_list)

In [6]:
image_size

3152

In [7]:
label_list

['Peach___Bacterial_spot',
 'Peach___Bacterial_spot',
 'Peach___Bacterial_spot',
 'Peach___Bacterial_spot',
 'Peach___Bacterial_spot',
 'Peach___Bacterial_spot',
 'Peach___Bacterial_spot',
 'Peach___Bacterial_spot',
 'Peach___Bacterial_spot',
 'Peach___Bacterial_spot',
 'Peach___Bacterial_spot',
 'Peach___Bacterial_spot',
 'Peach___Bacterial_spot',
 'Peach___Bacterial_spot',
 'Peach___Bacterial_spot',
 'Peach___Bacterial_spot',
 'Peach___Bacterial_spot',
 'Peach___Bacterial_spot',
 'Peach___Bacterial_spot',
 'Peach___Bacterial_spot',
 'Peach___Bacterial_spot',
 'Peach___Bacterial_spot',
 'Peach___Bacterial_spot',
 'Peach___Bacterial_spot',
 'Peach___Bacterial_spot',
 'Peach___Bacterial_spot',
 'Peach___Bacterial_spot',
 'Peach___Bacterial_spot',
 'Peach___Bacterial_spot',
 'Peach___Bacterial_spot',
 'Peach___Bacterial_spot',
 'Peach___Bacterial_spot',
 'Peach___Bacterial_spot',
 'Peach___Bacterial_spot',
 'Peach___Bacterial_spot',
 'Peach___Bacterial_spot',
 'Peach___Bacterial_spot',
 

Transform Image Labels uisng Scikit Learn's LabelBinarizer

In [8]:
label_binarizer = LabelBinarizer()
image_labels = label_binarizer.fit_transform(label_list)
pickle.dump(label_binarizer, open('label_transform.pkl', 'wb'))
n_classes = len(label_binarizer.classes_)

In [9]:
print(label_binarizer.classes_)

['Peach___Bacterial_spot' 'Pepper__bell___Bacterial_spot'
 'Pepper__bell___healthy' 'Potato___Early_blight' 'Potato___Late_blight'
 'Potato___healthy' 'Tomato_Bacterial_spot' 'Tomato_Early_blight'
 'Tomato_Late_blight' 'Tomato_Leaf_Mold' 'Tomato_Septoria_leaf_spot'
 'Tomato_Spider_mites_Two_spotted_spider_mite' 'Tomato__Target_Spot'
 'Tomato__Tomato_YellowLeaf__Curl_Virus' 'Tomato__Tomato_mosaic_virus'
 'Tomato_healthy']


In [10]:
np_image_list = np.array(image_list, dtype=np.float16) / 225.0

Splitting the Data into train data and test data

In [11]:
x_train, x_test, y_train, y_test = train_test_split(np_image_list, image_labels, test_size=0.2, random_state=42)

In [12]:
aug = ImageDataGenerator(rotation_range=25, width_shift_range=0.1, height_shift_range=0.1, shear_range=0.2, zoom_range=0.2, horizontal_flip=True,
                         fill_mode="nearest")

__Model Creation__

In [13]:
model = Sequential()
inputShape = (height, width, depth)
chanDim = -1
if K.image_data_format == "channels_first":
  inputShape = (depth, height, width)
  chanDim = 1
model.add(Conv2D(32, (3, 3), padding="same", input_shape=inputShape))
model.add(Activation("relu"))
model.add(BatchNormalization(axis=chanDim))
model.add(MaxPooling2D(pool_size=(3, 3)))
model.add(Dropout(0.25))
model.add(Conv2D(64, (3, 3), padding="same"))
model.add(Activation("relu"))
model.add(BatchNormalization(axis=chanDim))
model.add(Conv2D(64, (3, 3), padding="same"))
model.add(Activation("relu"))
model.add(BatchNormalization(axis=chanDim))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Conv2D(128, (3, 3), padding="same"))
model.add(Activation("relu"))
model.add(BatchNormalization(axis=chanDim))
model.add(Conv2D(128, (3, 3), padding="same"))
model.add(Activation("relu"))
model.add(BatchNormalization(axis=chanDim))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(1024))
model.add(Activation("relu"))
model.add(BatchNormalization())
model.add(Dropout(0.5))
model.add(Dense(n_classes))
model.add(Activation("softmax"))

In [14]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 256, 256, 32)      896       
_________________________________________________________________
activation (Activation)      (None, 256, 256, 32)      0         
_________________________________________________________________
batch_normalization (BatchNo (None, 256, 256, 32)      128       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 85, 85, 32)        0         
_________________________________________________________________
dropout (Dropout)            (None, 85, 85, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 85, 85, 64)        18496     
_________________________________________________________________
activation_1 (Activation)    (None, 85, 85, 64)        0

In [15]:
opt = Adam(lr = 0.001, decay = 0.001 / EPOCHS)
model.compile(loss="categorical_crossentropy", optimizer = opt, metrics = ["accuracy"])

In [ ]:
history = model.fit_generator(aug.flow(x_train, y_train, batch_size=BS), validation_data=(x_test, y_test),
                              epochs = EPOCHS)


C:\Users\samya\anaconda3\lib\site-packages\tensorflow\python\keras\engine\training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/25
79/79 [==============================] - 299s 4s/step - loss: 2.7211 - accuracy: 0.3165 - val_loss: 23.3648 - val_accuracy: 0.0555
Epoch 2/25
79/79 [==============================] - 270s 3s/step - loss: 1.6712 - accuracy: 0.5243 - val_loss: 13.9899 - val_accuracy: 0.0713
Epoch 3/25
79/79 [==============================] - 267s 3s/step - loss: 1.3915 - accuracy: 0.5849 - val_loss: 16.8900 - val_accuracy: 0.0745
Epoch 4/25
79/79 [==============================] - 266s 3s/step - loss: 1.4008 - accuracy: 0.5830 - val_loss: 11.5860 - val_accuracy: 0.1363
Epoch 5/25
79/79 [==============================] - 273s 3s/step - loss: 1.0026 - accuracy: 0.6988 - val_loss: 5.8840 - val_accuracy: 0.1933
Epoch 6/25
79/79 [==============================] - 267s 3s/step - loss: 0.8786 - accuracy: 0.7336 - val_loss: 7.2361 - val_accuracy: 0.2092
Epoch 7/25
79/79 [==============================] - 271s 3s/step - loss: 0.8613 - accuracy: 0.7293 - val_loss: 5.5477 - val_accuracy: 0.2266
Epoch 8/2